## Library 호출 및 path 설정

In [2]:
import pandas as pd
import numpy as np
import json
import requests
import unicodedata
from copy import deepcopy

path = "https://raw.githubusercontent.com/Algorithm-Coding-Test-Data-Analysis/algoview/main/dataAnalysis/macro/data.json"                
resp = requests.get(path)

## 예외 처리

In [3]:
def unicode_err(df):  # 회사명, 문제유형, 문제이름 중복 오류
    
    '''
    unicodedata.normalize() 함수는 문자열에 있는 유니코드 문자를 
    NFKC형식(문자열을 표준 호환성(KC) 형식으로 정규화)으로 정규화합니다.
    problem_name, problem_type, company_name 컬럼의 값 중 같은 값을 
    파이썬에서 다르게 인식하는 문제를 해결합니다.
    
    Args:
        data : (DataFrame | Series)

    Returns:
        data : (DataFrame | Series)
    '''
    
    df["company_name"] = df['company_name'].apply(lambda x: unicodedata.normalize('NFKC', x))
    df["problem_type"] = df['problem_type'].apply(lambda x: unicodedata.normalize('NFKC', x))
    return df

def prob_type_to_etc(value):  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    '''
    오류로 인해 실제 문제 유형이 아닌 값들을 예외 처리
    추후 추가 혹은 삭제 될 수 있습니다.
    
    Args:
        data : (DataFrame | Series)

    Returns:
        data : (DataFrame | Series)
    '''
    
    err_lst = ['하은.js', '문자열과', '별찍기', '개수', '다이나믹', '준혁.js', 
               '에라토스테네스의체', '수정.js' ,'지현.js', '출력하기', '회전시키기', 
               '판별하기', '반복', '반복해서', '바꿔서', '돌리기', '숫자의', '구하기']
    if value in err_lst:
        return "기타"
    else:
        return value
    
    
def file_name_exception(json_file):
    err_lst = ["Lv1_프로그래머스_행렬의덧셈_구현_지현.js",  # 1차 cycle 예외 처리 (파일명 오류, 개선 될 예정)
               "Lv0_0000_뒤에서 5등까지_구현_수정.js",
               "Lv0_0000_특정 문자열로 끝나는 가장 긴 부분 문자열 찾기_구현_수정.js",
               "Lv0_0000_자릿수더하기_구현_지현.js"]

    print("예외 처리 전 json 파일 길이: ", len(json_file))
    for error_file in err_lst:  # 1차 cycle 예외 처리 (파일명 오류, 개선 될 예정)
        del json_file[error_file]
    print("예외 처리 후 json 파일 길이: ", len(json_file))
    
    return json_file


## DataFrame으로 변환

In [6]:
def split_data_into_py_js(data, lang = "py"):  # Python, JavaScript로 data 분리
    
    '''
    전체 데이터인 DataFrame을 입력 받아
    Pytho과 JavaScript 별로 데이터를 분리

    Args:
        data (DataFrame): DataFrame으로 변환된 data
        language  (str): 언어 구분

    Returns:
        언어별 데이터 (DataFrame)
    '''
    
    print("-"*5, "split into python DataFrame & js DataFrame","-"*5)
    
    df = data.copy()
    if lang == "py":
        py_df = df[df["file_name"].str.contains(".py")].reset_index(drop = True)
        print("python DataFrame 길이: {0}".format(len(py_df)))
        return py_df
    
    elif lang == "js":
        js_df = df[df["file_name"].str.contains(".js")].reset_index(drop = True)
        print("js DataFrame 길이: {0}".format(len(js_df)))
        return js_df

##-----------------------------------------------------------------------------------------------------------------
    
def json_to_df(json_file):  # json_normalize 사용 불가, 따라서 methodcount 와 method명을 분리하기 위함
    
    '''
    json 형태인 데이터를 입력 받아
    DataFrame으로 변환
    ‘function_method’ column 안에 method 와 module이 함께 집계됩니다.

    Args:
        data : (json)

    Returns:
        data : (DataFrame)
    '''
    
    data_json = deepcopy(json_file)
    df = pd.DataFrame() # empty DataFrame
    for file_name in data_json.keys():
        tmp_df = pd.DataFrame(data_json[file_name])
        tmp_df = tmp_df.reset_index()
        df = pd.concat([df, tmp_df], axis = 0) 

    df = df.reset_index(drop = True).rename(columns = {"index" : "function_method"})  # method명 column으로 변환
    df = df[['file_name', 'level', 'year', 'company_name','problem_name', 'problem_type',  # column 순서 재배치
             'function_method', 'countmethod', 'module', 'line_count', 'code']]

    df = df.fillna(np.nan)
    df = unicode_err(df) # 회사명, 문제유형, 문제이름 중복 오류
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    return df

##-----------------------------------------------------------------------------------------------------------------


def json_to_df_split_module_method(json_file):  # module, method 별도의 column으로 분리
        
    '''
    json 형태인 데이터를 입력 받아
    DataFrame으로 변환
    module 과 method를 별도의 column으로 분리
    
    Args:
        data : (json)

    Returns:
        data : (DataFrame)
    '''
    
    data = deepcopy(json_file)
    data_2 = deepcopy(json_file)
    
    for key in data.keys():
        del data[key]["module"]
    for key in data_2.keys():
        del data_2[key]["countmethod"]

    method_df = pd.DataFrame()
    for key in data.keys():
        if data[key]["countmethod"] == 0:
            data[key]["countmethod"] = {}
        tmp_df = pd.DataFrame(data[key]).reset_index()
        tmp_df = tmp_df.rename(columns = {"index":"method"})
        method_df = pd.concat([method_df, tmp_df], axis = 0) 
    del tmp_df

    module_df = pd.DataFrame()
    for key in data_2.keys():
        tmp_df = pd.DataFrame(data_2[key]).reset_index()
        tmp_df = tmp_df.rename(columns = {"index":"module_name"})
        module_df = pd.concat([module_df, tmp_df], axis = 0)
        module_df = module_df.drop(module_df.columns.to_list()[2:], axis = 1)
    
    
    df = method_df.merge(module_df, on = "file_name", how = "left")
    df = df[['file_name', 'level', 'year', 'company_name', 'problem_name',  # column 순서 재배치
             'problem_type', 'method',  'countmethod', 'module_name', 'line_count', 'code']]
    
    df = df.fillna(np.nan)
    df = unicode_err(df)  # 회사명, 문제유형, 문제이름 중복 오류
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류

    return df


##-----------------------------------------------------------------------------------------------------------------

def json_to_df_dict():

    df = pd.read_json(path)
    df = df.T.reset_index(drop = True)
    df = df.fillna(np.nan)
    df = unicode_err(df)
    df["problem_type"] = df["problem_type"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    df["company_name"] = df["company_name"].apply(lambda x: prob_type_to_etc(x))  # 문제 유형이 아닌 카테고리를 "기타"로 분류
    
    return df 


### 하나의 column 안에 method 와 module 

In [7]:
data_json = json.loads(resp.text)
data_json = file_name_exception(data_json)
data_df = json_to_df(data_json)
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

예외 처리 전 json 파일 길이:  543
예외 처리 후 json 파일 길이:  539
----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 498
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 634


In [8]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,function_method,countmethod,module,line_count,code
0,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,print,1.0,NaN,8,"def solution(array, commands):\n answer = [..."
1,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,sorted,2.0,NaN,8,"def solution(array, commands):\n answer = [..."
2,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,.append,1.0,NaN,8,"def solution(array, commands):\n answer = [..."
3,Lv1_0000_프로그래머스_나머지가1이되는수찾기_구현_재호.py,level1,0000,프로그래머스,나머지가1이되는수찾기,구현,min,1.0,NaN,2,def solution(n):\n return min([i for i in r...
4,Lv1_0000_프로그래머스_나머지가1이되는수찾기_구현_재호.py,level1,0000,프로그래머스,나머지가1이되는수찾기,구현,range,1.0,NaN,2,def solution(n):\n return min([i for i in r...


In [9]:
py_df[py_df["file_name"] == "Lv1_2018_프로그래머스_소수 만들기_구현_재영.py"]

,file_name,level,year,company_name,problem_name,problem_type,function_method,countmethod,module,line_count,code
70,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,NaN,12,from itertools import combinations\ndef soluti...
71,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,NaN,12,from itertools import combinations\ndef soluti...
72,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,itertools,NaN,1.0,12,from itertools import combinations\ndef soluti...
73,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,combinations,NaN,1.0,12,from itertools import combinations\ndef soluti...


---

### method, module columns 분리

In [10]:
data_json = json.loads(resp.text)
data_json = file_name_exception(data_json)
data_df = json_to_df_split_module_method(data_json)
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

예외 처리 전 json 파일 길이:  543
예외 처리 후 json 파일 길이:  539
----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 516
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 634


In [11]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,method,countmethod,module_name,line_count,code
0,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,.append,1.0,NaN,8,"def solution(array, commands):\n answer = [..."
1,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,print,1.0,NaN,8,"def solution(array, commands):\n answer = [..."
2,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,sorted,2.0,NaN,8,"def solution(array, commands):\n answer = [..."
3,Lv1_0000_프로그래머스_나머지가1이되는수찾기_구현_재호.py,level1,0000,프로그래머스,나머지가1이되는수찾기,구현,min,1.0,NaN,2,def solution(n):\n return min([i for i in r...
4,Lv1_0000_프로그래머스_나머지가1이되는수찾기_구현_재호.py,level1,0000,프로그래머스,나머지가1이되는수찾기,구현,range,1.0,NaN,2,def solution(n):\n return min([i for i in r...


In [12]:
py_df[py_df["file_name"] == "Lv1_2018_프로그래머스_소수 만들기_구현_재영.py"]

,file_name,level,year,company_name,problem_name,problem_type,method,countmethod,module_name,line_count,code
68,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,combinations,12,from itertools import combinations\ndef soluti...
69,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,range,1.0,itertools,12,from itertools import combinations\ndef soluti...
70,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,combinations,12,from itertools import combinations\ndef soluti...
71,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,sum,1.0,itertools,12,from itertools import combinations\ndef soluti...


---

### dict 형태의 column

In [13]:
data_df = json_to_df_dict()
py_df = split_data_into_py_js(data_df, "py")
js_df = split_data_into_py_js(data_df, "js")

----- split into python DataFrame & js DataFrame -----
python DataFrame 길이: 241
----- split into python DataFrame & js DataFrame -----
js DataFrame 길이: 302


In [14]:
py_df.head()

,file_name,level,year,company_name,problem_name,problem_type,countmethod,line_count,module,code
0,Lv1_0000_프로그래머스_K번째수_구현_재영.py,level1,0000,프로그래머스,K번째수,구현,"{'print': 1, 'sorted': 2, '.append': 1}",8,{},"def solution(array, commands):\n answer = [..."
1,Lv1_0000_프로그래머스_나머지가1이되는수찾기_구현_재호.py,level1,0000,프로그래머스,나머지가1이되는수찾기,구현,"{'min': 1, 'range': 1}",2,{},def solution(n):\n return min([i for i in r...
2,Lv1_0000_프로그래머스_약수의개수와덧셈_구현_재호.py,level1,0000,프로그래머스,약수의개수와덧셈,구현,"{'range': 1, 'int': 1}",9,{},"def solution(left, right):\n result = 0\n ..."
3,Lv1_2021_카카오_숫자문자열과영단어_구현_선빈.py,level1,2021,카카오,숫자문자열과영단어,구현,"{'.items': 1, '.compile': 1, '.sub': 1, 'str':...",18,{'re': 1},import re\ndef solution(s):\n table = {\n ...
4,Lv1_0000_프로그래머스_수박수박수박수박수박수__구현_재호.py,level1,0000,프로그래머스,수박수박수박수박수박수,,{'range': 1},8,{},def solution(n):\n result = ''\n for i i...


In [15]:
py_df[py_df["file_name"] == "Lv1_2018_프로그래머스_소수 만들기_구현_재영.py"]

,file_name,level,year,company_name,problem_name,problem_type,countmethod,line_count,module,code
33,Lv1_2018_프로그래머스_소수 만들기_구현_재영.py,level1,2018,프로그래머스,소수 만들기,구현,"{'sum': 1, 'range': 1}",12,"{'itertools': 1, 'combinations': 1}",from itertools import combinations\ndef soluti...
